# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [57]:
stocks = pd.read_csv("sp_500_stocks.csv")
from my_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [58]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2321237697660, 'week52high': 178.3, 'week52low': 123.98, 'week52highSplitAdjustOnly': 179.61, 'week52lowSplitAdjustOnly': 124.17, 'week52change': -0.10351797852500566, 'sharesOutstanding': 15821946000, 'float': 0, 'avg10Volume': 54577302, 'avg30Volume': 70612566, 'day200MovingAvg': 146.8, 'day50MovingAvg': 144.78, 'employees': 164000, 'ttmEPS': 5.88, 'ttmDividendRate': 0.9170023129162208, 'dividendYield': 0.006250441775722314, 'nextDividendDate': '', 'exDividendDate': '2023-02-10', 'nextEarningsDate': '2023-05-04', 'peRatio': 24.39017870632861, 'beta': 1.3052200029310859, 'maxChangePercent': 55.96148470259357, 'year5ChangePercent': 2.511976291586673, 'year2ChangePercent': 0.228724910008526, 'year1ChangePercent': -0.10351797852500566, 'ytdChangePercent': 0.1308706648095035, 'month6ChangePercent': -0.13330891561709868, 'month3ChangePercent': -0.006687980457393383, 'month1ChangePercent': 0.03576791625748288, 'day30ChangePercent': 0.020658756578695

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [8]:
data["year1ChangePercent"]

-0.10351797852500566

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [51]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [86]:
final_dataFrame = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url= f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        year1Change = data[symbol]['stats']['year1ChangePercent']
        price =  data[symbol]['quote']['latestPrice']
        stock = pd.DataFrame([[symbol,price,year1Change,"N/A"]],columns=my_columns)
        final_dataFrame = pd.concat([final_dataFrame,stock],ignore_index=True)
final_dataFrame

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,142.18,0.062806,N/A
1,AAL,15.63,-0.10882,N/A
2,AAP,140.63,-0.300243,N/A
3,AAPL,147.92,-0.103518,N/A
4,ABBV,154.28,0.06007,N/A
...,...,...,...,...
496,YUM,126.57,0.040014,N/A
497,ZBH,123.30,-0.023494,N/A
498,ZBRA,296.23,-0.301744,N/A
499,ZION,50.39,-0.27042,N/A


In [95]:
# final_dataFrame = pd.DataFrame(columns=my_columns)
# for symbol in symbol_string.split(","):
#     year1Change = data[symbol]['stats']['year1ChangePercent']
#     price =  data[symbol]['quote']['latestPrice']
#     stock = pd.DataFrame([[symbol,price,year1Change,"N/A"]],columns=my_columns)
#     final_dataFrame = pd.concat([final_dataFrame,stock],ignore_index=True)
# final_dataFrame
# Just 1 loop --> was for testing

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [106]:
final_dataFrame.sort_values('One-Year Price Return', ascending=False, inplace=True)
top50Stock = final_dataFrame[:50].reset_index()
top50Stock

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,193,FTI,15.67,1.238235,N/A
2,313,MPC,126.58,0.672054,N/A
3,457,UNM,44.26,0.62108,N/A
4,465,VLO,135.05,0.613582,N/A
5,463,VAR,177.07,0.571162,N/A
6,225,HRB,37.07,0.563488,N/A
7,355,OXY,58.96,0.535186,N/A
8,271,KSU,293.59,0.506512,N/A
9,289,LW,101.03,0.492134,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [111]:
def getPortfolioSize():
    global portfolio_size
    portfolio_size = input("Please Input your Portfolio Size")
    try:
        val = float(portfolio_size)
    except ValueError:
        portfolio_size=input('Please enter a number:')
        val = float(portfolio_size)

In [120]:
getPortfolioSize()

Please Input your Portfolio Size100000
100000.0


In [136]:
position_size = float(portfolio_size)/len(top50Stock.index)
for i in range(0,len(top50Stock)):
    numStockToBuy = math.floor(position_size/top50Stock.loc[i,"Price"])
    top50Stock.loc[i, 'Number of Shares to Buy']=numStockToBuy
top50Stock

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,25
1,193,FTI,15.67,1.238235,127
2,313,MPC,126.58,0.672054,15
3,457,UNM,44.26,0.62108,45
4,465,VLO,135.05,0.613582,14
5,463,VAR,177.07,0.571162,11
6,225,HRB,37.07,0.563488,53
7,355,OXY,58.96,0.535186,33
8,271,KSU,293.59,0.506512,6
9,289,LW,101.03,0.492134,19


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: